# 챗봇

## 루브릭
1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?
```python
def build_corpus(df, length, labels):
    from konlpy.tag import Mecab

    tokenizer = Mecab(dicpath=r"C:\mecab\share\mecab-ko-dic")
    df_copy = df.copy()
    # 각 레이블에 전처리, 토큰화, 길이제한 적용
    for l in labels:
        df_copy[l] = df_copy[l].apply(lambda x: preprocessing(x))
        df_copy[l] = df_copy[l].apply(lambda x: tokenizer.morphs(x))
        df_copy = df_copy[df_copy[l].apply(len) <= length]
    removed_num = df.shape[0] - df_copy.shape[0]
    print(
        f"데이터 길이 제한으로 사라진 데이터 수 : {removed_num}, {(removed_num)/df.shape[0]*100}%"
    )
    # 중복 제거
    df_copy = df_copy.drop_duplicates(labels[0])
    df_copy = df_copy.drop_duplicates(labels[1])

    print(
        f"데이터 중복으로 사라진 데이터 수 : {df.shape[0]-df_copy.shape[0]-removed_num}, {(df.shape[0]-df_copy.shape[0]-removed_num)/df.shape[0]*100}%"
    )
    return df_copy, tokenizer
```
2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?
```text
...
poch  9: 100%|██████████| 213/213 [00:05<00:00, 40.80it/s, Loss 0.7520]
Valid Loss 3.0349
Epoch 10: 100%|██████████| 213/213 [00:05<00:00, 40.81it/s, Loss 0.6610]
Valid Loss 2.9380
```

3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?
```text
Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 아 하 는 걸 만나 봐요 . <end>

Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 그렇 기 도 해요 . <end>
```

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import random
from sklearn.model_selection import train_test_split
import nltk
import gensim

from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

## 데이터 

https://github.com/songys/Chatbot_data

In [2]:
df = pd.read_csv("data/ChatbotData.csv")

In [3]:
df = df.drop("label", axis=1)

### 전처리, 토큰화, 중복제거

In [4]:
# 중복 질문 확인
df[(df.duplicated("Q") == True) | (df.duplicated("A") == True)]

,Q,A
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.
12,가끔 뭐하는지 궁금해,그 사람도 그럴 거예요.
18,가스불 켜놓고 나온거 같아,빨리 집에 돌아가서 끄고 나오세요.
...,...,...
11807,화이트데이에 고백할까요?,선물을 주면서 솔직하고 당당하게 고백해보세요.
11809,확실히 날 좋아하는 걸 아는 남자랑 친구가 될 수 있을까?,그 사람을 위해서는 그러면 안돼요.
11816,회식하는데 나만 챙겨줘. 썸임?,호감이 있을 수도 있어요. 그렇지만 조금 더 상황을 지켜보세요.
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.


In [6]:
df[df["Q"] == "고양이 키우고 싶어"]

,Q,A
195,고양이 키우고 싶어,자신을 먼저 키우세요.
196,고양이 키우고 싶어,가족들과 상의해보세요.


In [7]:
def preprocessing(sentence):
    import re

    sentence = sentence.lower()  # 소문자화
    sentence = re.sub(r"[^가-힣1-9a-zA-Z?.!,]+", " ", sentence)  # 기타 문자 제거

    return sentence

In [8]:
def build_corpus(df, length, labels):
    from konlpy.tag import Mecab

    tokenizer = Mecab(dicpath=r"C:\mecab\share\mecab-ko-dic")
    df_copy = df.copy()
    # 각 레이블에 전처리, 토큰화, 길이제한 적용
    for l in labels:
        df_copy[l] = df_copy[l].apply(lambda x: preprocessing(x))
        df_copy[l] = df_copy[l].apply(lambda x: tokenizer.morphs(x))
        df_copy = df_copy[df_copy[l].apply(len) <= length]
    removed_num = df.shape[0] - df_copy.shape[0]
    print(
        f"데이터 길이 제한으로 사라진 데이터 수 : {removed_num}, {(removed_num)/df.shape[0]*100}%"
    )
    # 중복 제거
    df_copy = df_copy.drop_duplicates(labels[0])
    df_copy = df_copy.drop_duplicates(labels[1])

    print(
        f"데이터 중복으로 사라진 데이터 수 : {df.shape[0]-df_copy.shape[0]-removed_num}, {(df.shape[0]-df_copy.shape[0]-removed_num)/df.shape[0]*100}%"
    )
    return df_copy, tokenizer

In [9]:
df_clean, mecab = build_corpus(df, 20, df.columns[:2])

데이터 길이 제한으로 사라진 데이터 수 : 151, 1.277171614649412%
데이터 중복으로 사라진 데이터 수 : 4110, 34.762750570921085%


In [10]:
df_clean.shape

(7562, 2)

### 데이터 증강

사전 훈련된 wor2vec을 이용한 Lexical Substitution  

In [11]:
from gensim.models.keyedvectors import Word2VecKeyedVectors

word_vectors = Word2VecKeyedVectors.load("data/word2vec_ko.model")
wv = word_vectors.wv

In [285]:
# non 오류 찾기
def find_nan(sample_sentence, wv):

    # . , ? ! 등 기호가 대체되지 않게 단어만 선택
    valid_tokens = [tok for tok in sample_sentence if tok.isalnum()]

    selected_tok = random.choice(valid_tokens)

    result = []
    for tok in sample_sentence:
        if tok is selected_tok:
            try:
                print(tok, wv.most_similar(tok)[0][0])
            except:
                print(tok, "사전에 없음")

In [292]:
wv.most_similar("비싼데")

[('假裝', 0.6139413118362427),
 ('칠검', 0.6112638115882874),
 ('빛난', 0.6083515286445618),
 ('큰데', 0.603387176990509),
 ('돋보인', 0.6031648516654968),
 ('가까워서', 0.6004668474197388),
 ('심했었', 0.5907561182975769),
 ('적다', 0.5850600004196167),
 ('家長', 0.5813945531845093),
 ('슬퍼했으며', 0.5703328251838684)]

In [293]:
# df_clean["Q"][:100].apply(lambda x: find_nan(x, wv))

In [12]:
def lexical_sub(sample_sentence, wv):

    # . , ? ! 등 기호가 대체되지 않게 단어만 선택
    valid_tokens = [tok for tok in sample_sentence if tok.isalnum()]

    selected_tok = random.choice(valid_tokens)

    result = []
    for tok in sample_sentence:
        if tok is selected_tok:
            try:
                result.append(wv.most_similar(tok)[0][0])
            except:
                result.append(tok)
        else:
            result.append(tok)
    return result

In [13]:
# 데이터 증강 
df_aug = pd.DataFrame()
df_aug["Q"] = df_clean["Q"].apply(lambda x: lexical_sub(x, wv))
df_aug["A"] = df_clean["A"].apply(lambda x: lexical_sub(x, wv))

df_combined = pd.concat([df_clean, df_aug], ignore_index=True)

In [14]:
# 증강된 데이터 비교 
df_aug["Q"][0], df_clean["Q"][0]

(['12', '시경', '땡', '!'], ['12', '시', '땡', '!'])

In [15]:
# 2만개 가량으로 증강
df_combined.shape

(15124, 2)

### 벡터화

In [16]:
df_combined["A"] = df_combined["A"].apply(lambda x: ["<start>"] + x + ["<end>"])
# df_clean["A"] = df_clean["A"].apply(lambda x: ["<start>"] + x + ["<end>"])

In [17]:
def tokenize(df, NUM_WORDS):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS, filters="")
    corpus = []
    for _, sentence in df.iterrows():
        corpus.extend(sentence[df.columns[0]])
        corpus.extend(sentence[df.columns[1]])
    tokenizer.fit_on_texts(corpus)

    temp_df = df.copy()
    temp_df["Q"] = tokenizer.texts_to_sequences(df["Q"])
    temp_df["A"] = tokenizer.texts_to_sequences(df["A"])
    max_len = max(temp_df["Q"].apply(len).max(), temp_df["A"].apply(len).max())

    Q = tf.keras.preprocessing.sequence.pad_sequences(
        temp_df["Q"], maxlen=max_len, padding="post"
    )
    A = tf.keras.preprocessing.sequence.pad_sequences(
        temp_df["A"], maxlen=max_len, padding="post"
    )

    return Q, A, tokenizer

In [18]:
enc, dec, tokenizer = tokenize(df_combined, None)
max_len = enc.shape[1]
# 총 단어 개수
len(tokenizer.word_index)

7545

In [19]:
# 증강하지 않은 데이터

# enc_clean, dec_clean, tokenizer_clean = tokenize(df_clean, None)
# max_len_clean = enc_clean.shape[1]
# # 총 단어 개수
# len(tokenizer_clean.word_index)

6190

### 데이터 분할

In [20]:
# 분할
enc_train, enc_test, dec_train, dec_test = train_test_split(
    enc, dec, test_size=0.1, random_state=42
)

In [21]:
enc_train.shape, enc_test.shape, dec_train.shape, dec_test.shape

((13611, 22), (1513, 22), (13611, 22), (1513, 22))

### 데이터 세트로 변형

In [23]:
BATCH_SIZE = 64
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).batch(
    batch_size=BATCH_SIZE
)

In [221]:
dataset_valid = tf.data.Dataset.from_tensor_slices((enc_test, dec_test)).batch(
    batch_size=BATCH_SIZE
)

In [22]:
# 증강하지 않은 데이터

# BATCH_SIZE = 64
# dataset_clean = tf.data.Dataset.from_tensor_slices(
#     (
#         enc_clean,
#         dec_clean,
#     )
# ).batch(batch_size=BATCH_SIZE)

## 모델

[트랜스포머 모델](../../model/transformer.py)

In [25]:
import sys
import os

current_dir = os.path.dirname(os.path.abspath("chatbot.ipynb"))
model_dir = os.path.abspath(os.path.join(current_dir, "../../model"))
sys.path.append(model_dir)

import transformer

In [222]:
# 외부 모듈 수정하면 다시 불러오는 거
import importlib



importlib.reload(transformer)

<module 'transformer' from 'c:\\Users\\zzoon\\projects\\aiffel_camp\\model\\transformer.py'>

In [223]:
example_sentence = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야.",
]

In [224]:
# num_words가 None인 무제한인 경우 단어 사전의 길이로 사전의 크기를 정의
if tokenizer.num_words:


    VOCAB_SZIE = tokenizer.num_words

else:
    VOCAB_SZIE = len(tokenizer.word_index)



model = transformer.Transformer(
    n_layers=2,

    d_model=256,
    n_heads=8,
    d_ff=512,

    src_vocab_size=VOCAB_SZIE,
    tgt_vocab_size=VOCAB_SZIE,
    pos_len=max_len,
)

In [226]:
importlib.reload(transformer)
transformer.train(
    model,
    dataset,

    EPOCH=10,
    enc_tokenizer=tokenizer,
    dec_tokenizer=tokenizer,
    example_sentence=example_sentence,
    type="mecab",
    show_translate=True,
    plot_attention=False,
    valid_data=dataset_valid,
)

Epoch  1: 100%|██████████| 213/213 [00:12<00:00, 17.19it/s, Loss 3.3729]


Valid Loss 3.6366
Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 은 사람 이 있 는 것 도 좋 을 거 예요 . <end>
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 잘 하 고 있 는 게 좋 은 거 예요 . <end>
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 좋 은 사람 이 있 었 나 봐요 . <end>
Input: 집에 있는다는 소리야.
Predicted translation: 많이 하 고 싶 은 아니 에요 . <end>


Epoch  2: 100%|██████████| 213/213 [00:05<00:00, 39.22it/s, Loss 3.2017]


Valid Loss 3.5129
Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 아 하 는 것 도 좋 아요 . <end>
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 잘 하 고 있 나 봐요 . <end>
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 좋 아 하 지 마세요 . <end>
Input: 집에 있는다는 소리야.
Predicted translation: 많이 하 고 싶 네요 . <end>


Epoch  3: 100%|██████████| 213/213 [00:05<00:00, 41.09it/s, Loss 2.8826]


...

Valid Loss 2.9818
Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 아 하 는 걸 만나 봐요 . <end>
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 그렇게 생각 하 게 될 거 예요 . <end>
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 이해 해 보 세요 . <end>
Input: 집에 있는다는 소리야.
Predicted translation: 집 에서 이랑 같이 있 는지 벗어나 세요 . <end>


Epoch  9: 100%|██████████| 213/213 [00:05<00:00, 40.80it/s, Loss 0.7520]


Valid Loss 3.0349
Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 아 먼저 생각 해 보 세요 . <end>
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 그것 일 함 이 남 죠 . <end>
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 조금 만 생각 해 보 세요 . <end>
Input: 집에 있는다는 소리야.
Predicted translation: 안 같이 있 고 싶 네요 . <end>


Epoch 10: 100%|██████████| 213/213 [00:05<00:00, 40.81it/s, Loss 0.6610]


Valid Loss 2.9380
Input: 지루하다, 놀러가고 싶어.
Predicted translation: 좋 는다면 후회 하 고 있 다면 먼저 먼저 말 해 보 세요 .
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 다음 에 는 잊어버리 는 게 습관 이 라고 생각 하 세요 . <end>
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 새로운 걸 표현 하 세요 . <end>
Input: 집에 있는다는 소리야.
Predicted translation: 세상 의 같이 듣 고 싶 네요 . <end>


## 평가

In [276]:
print(example_sentence[1])
transformer.translate_mecab().translate(
    example_sentence[1], model, tokenizer, tokenizer
)

오늘 일찍 일어났더니 피곤하다.


'다음 에 는 잊어버리 는 게 습관 이 라고 생각 하 세요 . <end>'

### 종합 bleu점수

In [250]:
# bleu 점수
def eval_bleu_single(
    model,
    src_sentence,
    tgt_sentence,
    src_tokenizer,
    tgt_tokenizer,
    tokeninze_type,
    max_len,
    verbose=True,
):
    from nltk.translate.bleu_score import sentence_bleu
    from nltk.translate.bleu_score import SmoothingFunction

    if tokeninze_type == "sp":
        src_tokens = src_tokenizer.encode_as_ids(src_sentence)
        tgt_tokens = tgt_tokenizer.encode_as_ids(tgt_sentence)
        translator = transformer.translate_sp()
    elif tokeninze_type == "mecab":
        src_tokens = src_tokenizer.texts_to_sequences(src_sentence.split(" "))
        tgt_tokens = tgt_tokenizer.texts_to_sequences(tgt_sentence.split(" "))

        translator = transformer.translate_mecab()

    if len(src_tokens) > max_len:
        return None
    if len(tgt_tokens) > max_len:
        return None

    reference = tgt_sentence.split()
    # 토큰 제거
    reference = reference[1:-1]

    candidate = translator.translate(
        src_sentence, model, src_tokenizer, tgt_tokenizer
    ).split()
    # 토큰 제거
    candidate = candidate[:-1]
    score = sentence_bleu(
        [reference], candidate, smoothing_function=SmoothingFunction().method1
    )

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real: ", reference)
        print("Score: %lf\n" % score)

    return score


def eval_bleu(
    model,
    src_sentences,
    tgt_sentence,
    src_tokenizer,
    tgt_tokenizer,
    tokeninze_type,
    max_len,
    verbose=True,
):
    total_score = 0.0
    sample_size = len(src_sentences)

    from tqdm import tqdm

    for idx in tqdm(range(sample_size)):
        score = eval_bleu_single(
            model,
            src_sentences[idx],
            tgt_sentence[idx],
            src_tokenizer,
            tgt_tokenizer,
            tokeninze_type,
            max_len,
            verbose,
        )
        if not score:
            continue

        total_score += score

    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)

In [50]:
enc_test_text = tokenizer.sequences_to_texts(enc_test)
dec_test_text = tokenizer.sequences_to_texts(dec_test)

In [251]:
eval_bleu_single(
    model, enc_test_text[0], dec_test_text[0], tokenizer, tokenizer, "mecab", 50, True
)

Source Sentence:  친구 도 연인 도 아닌 그런 사이 의 끝
Model Prediction:  ['후유증', '이', '클', '수', '도', '있', '어요', '.']
Real:  ['후유증', '이', '클', '수', '도', '있', '겠', '어요', '.']
Score: 0.675292



0.6752918218126556

In [277]:
eval_bleu(
    model,
    enc_test_text,
    dec_test_text,
    tokenizer,
    tokenizer,
    "mecab",
    100,
    verbose=False,
)

100%|██████████| 1513/1513 [16:26<00:00,  1.53it/s]

Num of Sample: 1513
Total Score: 0.08246267707338589


### Beam Search Decoder로 결과 확인

In [269]:
# beam_search_decoder() 구현
def beam_search_decoder(
    sentence, src_len, tgt_len, model, src_tokenizer, tgt_tokenizer, beam_size
):
    def calc_prob(src_ids, tgt_ids, model):
        enc_padding_mask, enc_dec_mask, dec_padding_mask = transformer.generate_masks(
            src_ids, tgt_ids
        )

        predictions, _, _, _ = model(
            src_ids, tgt_ids, enc_padding_mask, enc_dec_mask, dec_padding_mask
        )
        return tf.math.softmax(predictions, axis=-1)

    tokens = src_tokenizer.texts_to_sequences(sentence.split(" "))

    tokens = [x for token in tokens for x in token]

    src_in = tf.keras.preprocessing.sequence.pad_sequences(
        [tokens], maxlen=src_len, padding="post"
    )

    pred_cache = np.zeros((beam_size * beam_size, tgt_len), dtype=np.int64)
    pred_tmp = np.zeros((beam_size, tgt_len), dtype=np.int64)

    eos_flag = np.zeros((beam_size,), dtype=np.int64)
    scores = np.ones((beam_size,))

    pred_tmp[:, 0] = tgt_tokenizer.word_index["<start>"]

    dec_in = tf.expand_dims(pred_tmp[0, :1], 0)
    prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

    for seq_pos in range(1, tgt_len):
        score_cache = np.ones((beam_size * beam_size,))

        # init
        for branch_idx in range(beam_size):
            cache_pos = branch_idx * beam_size

            score_cache[cache_pos : cache_pos + beam_size] = scores[branch_idx]
            pred_cache[cache_pos : cache_pos + beam_size, :seq_pos] = pred_tmp[
                branch_idx, :seq_pos
            ]

        for branch_idx in range(beam_size):
            cache_pos = branch_idx * beam_size

            if seq_pos != 1:  # 모든 Branch를 로 시작하는 경우를 방지
                dec_in = pred_cache[branch_idx, :seq_pos]
                dec_in = tf.expand_dims(dec_in, 0)

                prob = calc_prob(src_in, dec_in, model)[0, -1].numpy()

            for beam_idx in range(beam_size):
                max_idx = np.argmax(prob)

                score_cache[cache_pos + beam_idx] *= prob[max_idx]
                pred_cache[cache_pos + beam_idx, seq_pos] = max_idx

                prob[max_idx] = -1

        for beam_idx in range(beam_size):
            if eos_flag[beam_idx] == -1:
                continue

            max_idx = np.argmax(score_cache)
            prediction = pred_cache[max_idx, : seq_pos + 1]

            pred_tmp[beam_idx, : seq_pos + 1] = prediction
            scores[beam_idx] = score_cache[max_idx]
            score_cache[max_idx] = -1

            if prediction[-1] == tgt_tokenizer.word_index["<end>"]:
                eos_flag[beam_idx] = -1

    pred = []
    for long_pred in pred_tmp:
        # zero_idx = long_pred.tolist().index(tgt_tokenizer.eos_id())
        # short_pred = long_pred[:zero_idx+1]
        # pred.append(short_pred)
        pred.append(long_pred)
    return pred

In [272]:
def beam(reference, ids, tokenizer):
    print("입력:", reference)
    reference = reference.split()

    for i, _id in enumerate(ids):
        # start, end 토큰과 패딩을 제외하고 한글로 변환
        sequence = [
            [x]
            for x in _id
            if x
            not in [tokenizer.word_index["<start>"], tokenizer.word_index["<end>"], 0]
        ]
        candidate = tokenizer.sequences_to_texts(sequence)
        candidate = " ".join(candidate)

        print(f"{i+1}번째 답변:", candidate)

In [270]:
ids = beam_search_decoder(
    enc_test_text[0],
    max_len,
    max_len,
    model,
    tokenizer,
    tokenizer,
    beam_size=5,
)

In [273]:
beam(enc_test_text[0], ids, tokenizer)

입력: 친구 도 연인 도 아닌 그런 사이 의 끝
1번째 답변: 후유증 이 클 수 도 있 어요 .
2번째 답변: 후유증 친구 클 수 도 있 어요 .
3번째 답변: 후유증 이 클 거 도 있 어요 .
4번째 답변: 친구 이 클 수 도 있 어요 .
5번째 답변: 후유증 친구 클 거 도 있 어요 .


In [275]:
# 원본
dec_test_text[0]

'<start> 후유증 이 클 수 도 있 겠 어요 . <end>'

## 결론 및 회고

모델 학습을 많이 돌려보지 않아서인지, valid loss는 3보다 조금 작을 정도 밖에 떨어지지 않았다. trainloss는 0.6까지 떨어지는 동안 과적합이 일어나지는 않았는데, 더 이상 valid가 떨어지기 힘들것 같았다. bleu는 번역이 아니고 챗봇이니까 별로 의미없다고 생각하였고, beam search가 여러 답변을 한번에 볼 수 있어서 재미있었다. 후유증을 친구랑 바꾼 것이 좋은 문장이라고 판단하는게 아주 어이가 없었지만, 지난번 챗봇보다는 괜찮다고 느낀게 일단 데이터에 있는 답변만 주지 않아서 좋았다.

데이터 중복을 제거할 때, 같은 질문에 여러 답이 나오는 것을 피하고 싶은지(a의 중복), 여러 질문에 한 답이 나오는 건지(q의 중복) 잘 모르겠어서 그냥 다 지워버리니까 4000개, 1/3의 데이터가 지워져서 의문이 들었다. 요구사항이 더 명확하게 나오면 좋을 것 같다. 사실 챗봇이니까 두 상황 모두 괜찮지 않을까 생각하였는데, 어떤 것을 의도하고 중복을 제거하라는 건지 명확한 이해가 되지 않았다.  

그래서 증강이 꼭 필요하였는데, lexical sub 방식은 운에 따라서 훈련 중 loss가 nan이 되버리는 현상이 있었다. 일단 lexical sub를 한 데이터가 그냥 중복으로 인식되어 그런 것 같았다. lexical을 더 많은 단어를 하거나 등의 여러 해결방안이 있었겠지만, 증강데이터를 다시 만드니 해결이 되었다. 랜덤으로 선택되는 단어가 문제가 있던거 같은데 뭐가 문제인지 정확하게 알지는 못하였다. 다른 분도 비슷한 오류가 난 걸 보니 이 데이터세트에는 lexical이 좋은 방법이 아닌 것 같았다. 